## Silent bystander edits for predicting pegRNA efficiency with PRIDICT2.0

Silent bystander edits can increase the efficiency of (e)pegRNA efficiencies. This notebook allows to create inputs for PRIDICT2.0 that contain silent bystander edits, up to 5 bp up- and downstream of the edit.

Requirements:
- PRIDICT input sequence with **150 bp** context on both sides of the edit (e.g. 150bp - (A/G) - 150bp; longer context needed than the 100bp for standard PRIDICT2.0)
- PRIDICT input sequence has to be *in frame* (ORF_start = 0) if the edit (and 5bp context up- and downstream of it) is within an exon. If not in an exon, still pretend it is "in frame" to run the scripts by keeping ORF_start = 0 (single) or setting the "in frame value" to "yes" (see batch input file).
- PRIDICT2.0 conda environment (includes necessary packages)

How to use:
- Use this notebook or the corresponding command line script, to create an input batch file for PRIDICT2.0 prediction with silent bystanders
- We only provide silent bystander predicitons with **1bp replacements** or **multibp replacements** (not with insertion/deletions)
- Input: PRIDICT input format, but with 150bp flanking bases on both sides
- Manual function: Create silent bystander PRIDICT inputs for 1 mutation/edit
- Batch function: Get the inputs for silent bystander for all PRIDICT inputs in an input .csv file
- Finally run PRIDICT2.0 (batch mode) with created input sequences to get efficiency predictions

### Import necessary packages

In [16]:
from Bio.Seq import Seq
import re
import pandas as pd
from itertools import product
import os

### Functions required to run notebook

In [19]:
def generate_all_sequences(length):
    ### Generate all possible sequences of a given length
    nucleotides = ['A', 'T', 'G', 'C']
    return [''.join(seq) for seq in product(nucleotides, repeat=length)]

def generate_combinations(left_context_close, left_options, edit, right_options, right_context_close):
    ### Generate all possible combinations of left and right context with edit
    return [
        left_context_close + left_opt + edit + right_opt + right_context_close
        for left_opt, right_opt in product(left_options, right_options)
    ]

def convert_differences_to_lowercase(option, original):
    ### Convert differences between option and original to lowercase
    return ''.join(
        opt.lower() if opt != orig else opt 
        for opt, orig in zip(option, original)
    )

def split_sequence(seq):
    ### Split a sequence into three parts: before first lowercase letter, between first and last lowercase letter, and after last lowercase letter
    lowercase_positions = [m.start() for m in re.finditer('[a-z]', seq)]
    if not lowercase_positions:
        return seq, '', '', 0
    
    first_lower_pos = lowercase_positions[0]
    last_lower_pos = lowercase_positions[-1]
    before_first_lower = seq[:first_lower_pos]
    between_first_last_lower = seq[first_lower_pos:last_lower_pos + 1]
    after_last_lower = seq[last_lower_pos + 1:]

    return before_first_lower, between_first_last_lower, after_last_lower, len(lowercase_positions)

def validate_context_length(input, minimum_flanking):
    ### Validate that the context length is at least the minimum flanking length
    left_context = len(input.split('(')[0])
    right_context = len(input.split(')')[1])
    if left_context < minimum_flanking or right_context < minimum_flanking:
        raise ValueError('Context length is less than minimum flanking length! Please check your input sequence.')

def process_contexts(input, bystander_window, close_context_len):
    ### Process the input sequence into left and right context
    left_context = input.split('(')[0]
    right_context = input.split(')')[1]
    left_context_start = left_context[:-bystander_window-close_context_len]
    left_context_close = left_context[-bystander_window-close_context_len:-bystander_window]
    left_bystander_window = left_context[-bystander_window:]
    right_context_end = right_context[bystander_window+close_context_len:]
    right_context_close = right_context[bystander_window:bystander_window+close_context_len]
    right_bystander_window = right_context[:bystander_window]
    return left_context_start, left_context_close, left_bystander_window, right_context_close, right_bystander_window, right_context_end

def create_silent_bystander_df(combinations_filtered,combinations_filtered_focus_only,combinations_filtered_AA, left_context_start, left_context_close, left_bystander_window, edit_before, right_bystander_window, right_context_close, right_context_end, name, edited_focus_sequence, edited_nobystander_focus_AA):
    ### Create a dataframe with the bystander sequences
    edit_name_bystander_lst = []
    pridict_input_lst = []
    original_length_lst = []
    final_length_lst = []
    totalbasechanges_lst = []
    editonly_focus_sequence_lst = []
    editonly_nobystander_focus_AA_lst = []
    edited_bystander_focus_sequence_lst = []
    edited_bystander_focus_AA_lst = []

    for ind, seq in enumerate(combinations_filtered):
        unchanged_before, edit, unchanged_after, totalbasechanges = split_sequence(seq)
        original_seq = left_context_close + left_bystander_window + edit_before + right_bystander_window + right_context_close
        original_seq = original_seq[len(unchanged_before):-len(unchanged_after)]
        final_pridict_seq = f"{left_context_start}{unchanged_before}({original_seq}/{edit}){unchanged_after}{right_context_end}"
        edit_name_bystander = f"{name}_{original_seq}_{edit}"

        edit_name_bystander_lst.append(edit_name_bystander)
        pridict_input_lst.append(final_pridict_seq)
        original_length_lst.append(len(edit_before))
        final_length_lst.append(len(edit))
        totalbasechanges_lst.append(totalbasechanges)
        editonly_focus_sequence_lst.append(edited_focus_sequence)
        editonly_nobystander_focus_AA_lst.append(edited_nobystander_focus_AA)
        edited_bystander_focus_sequence_lst.append(combinations_filtered_focus_only[ind])
        edited_bystander_focus_AA_lst.append(combinations_filtered_AA[ind])
        if edited_nobystander_focus_AA != combinations_filtered_AA[ind]:
            raise ValueError('Error in AA translation! Please contact developers.')



    return pd.DataFrame(list(zip(
        edit_name_bystander_lst, pridict_input_lst, original_length_lst, final_length_lst, totalbasechanges_lst, edited_bystander_focus_sequence_lst, edited_bystander_focus_AA_lst, editonly_focus_sequence_lst, editonly_nobystander_focus_AA_lst
    )), columns=['sequence_name', 'editseq', 'original_edit_length', 'final_edit_length_with_bystander', 'total_nr_of_base_changes', 'bystander_focus_sequence', 'bystander_focus_AA', 'editedonly_focus_sequence', 'editedonly_focus_AA'])

def isDNA(sequence):
    """ Check whether sequence contains only DNA bases. """
    onlyDNA = True
    diff_set = set(sequence) - set('ACTGatgc')
    if diff_set:
        onlyDNA = False
        print('Non-DNA bases detected. Please use ATGC.')
        print(sequence)
        raise ValueError
    return onlyDNA

def primesequenceparsing(sequence: str) -> object:
    """
    Function which takes target sequence with desired edit as input and 
    editing characteristics as output. Edit within brackets () and original
    equence separated by backslash from edited sequence: (A/G) == A to G mutation.
    Placeholder for deletions and insertions is '-'.

    Parameters
    ----------
    sequence : str
        Target sequence with desired edit in brackets ().
    """
    
    sequence = sequence.replace('\n','')  # remove any spaces or linebreaks in input
    sequence = sequence.replace(' ','')
    sequence = sequence.upper()
    if sequence.count('(') != 1:
        print(sequence)
        print('More or less than one bracket found in sequence! Please check your input sequence.')
        raise ValueError

    five_prime_seq = sequence.split('(')[0]
    three_prime_seq = sequence.split(')')[1]

    sequence_set = set(sequence)
    if '/' in sequence_set:
        original_base = sequence.split('/')[0].split('(')[1]
        edited_base = sequence.split('/')[1].split(')')[0]

        # edit flanking bases should *not* be included in the brackets
        if (original_base[0] == edited_base[0]) or (original_base[-1] == edited_base[-1]):
            print(sequence)
            print('Flanking bases should not be included in brackets! Please check your input sequence.')
            raise ValueError
    elif '+' in sequence_set:  #insertion
        original_base = '-'
        edited_base = sequence.split('+')[1].split(')')[0]
    elif '-' in sequence_set:  #deletion
        original_base = sequence.split('-')[1].split(')')[0]
        edited_base = '-'

    # ignore "-" in final sequences (deletions or insertions)
    if original_base == '-':
        original_seq = five_prime_seq + three_prime_seq
        if edited_base != '-':
            mutation_type = 'Insertion'
            correction_length = len(edited_base)
        else:
            print(sequence)
            raise ValueError
    else:
        original_seq = five_prime_seq + original_base + three_prime_seq
        if edited_base == '-':
            mutation_type = 'Deletion'
            correction_length = len(original_base)
        elif len(original_base) == 1 and len(edited_base) == 1:
            if isDNA(original_base) and isDNA(edited_base):  # check if only AGCT is in bases
                mutation_type = '1bpReplacement'
                correction_length = len(original_base)
            else:
                print(sequence)
                print('Non DNA bases found in sequence! Please check your input sequence.')
                raise ValueError
        elif len(original_base) > 1 or len(edited_base) > 1:
            if isDNA(original_base) and isDNA(edited_base):  # check if only AGCT is in bases
                mutation_type = 'MultibpReplacement'
                if len(original_base) == len(
                        edited_base):  # only calculate correction length if replacement does not contain insertion/deletion
                    correction_length = len(original_base)
                else:
                    print(sequence)
                    print('Only 1bp replacements or replacements of equal length (before edit/after edit) are supported! Please check your input sequence.')
                    raise ValueError
            else:
                print(sequence)
                print('Non DNA bases found in sequence! Please check your input sequence.')
                raise ValueError

    if edited_base == '-':
        edited_seq = five_prime_seq + three_prime_seq
    else:
        edited_seq = five_prime_seq + edited_base.lower() + three_prime_seq

    if isDNA(edited_seq) and isDNA(original_seq):  # check whether sequences only contain AGCT
        pass
    else:
        raise ValueError

    basebefore_temp = five_prime_seq[
                      -1:]  # base before the edit, could be changed with baseafter_temp if Rv strand is targeted (therefore the "temp" attribute)
    baseafter_temp = three_prime_seq[:1]  # base after the edit

    editposition_left = len(five_prime_seq)
    editposition_right = len(three_prime_seq)
    return original_base, edited_base, original_seq, edited_seq, editposition_left, editposition_right, mutation_type, correction_length, basebefore_temp, baseafter_temp

def bystander_creation_for_pridict(pridict_input_original, ORF_start, name, bystander_window, close_context_len, minimum_flanking):
    pridict_input_original = pridict_input_original.upper()
    validate_context_length(pridict_input_original, minimum_flanking)

    mutation_type = primesequenceparsing(pridict_input_original)[6]
    if not mutation_type in ['1bpReplacement', 'MultibpReplacement']:
        raise ValueError('Only 1bp replacements or multi bp replacements of equal length (before edit/after edit) are supported for silent bystander predictions! Please check your input sequence.')

    
    left_context_start, left_context_close, left_bystander_window, right_context_close, right_bystander_window, right_context_end = process_contexts(pridict_input_original, bystander_window, close_context_len)
    
    edit_before = pridict_input_original.split('(')[1].split('/')[0]
    edit_after = pridict_input_original.split('/')[1].split(')')[0].lower()
    
    left_bystander_options = [convert_differences_to_lowercase(option, left_bystander_window) 
                              for option in generate_all_sequences(bystander_window)]
    right_bystander_options = [convert_differences_to_lowercase(option, right_bystander_window) 
                               for option in generate_all_sequences(bystander_window)]
    
    combinations = generate_combinations(left_context_close, left_bystander_options, edit_after, right_bystander_options, right_context_close)
    edited_focus = f"{left_context_close}{left_bystander_window}{edit_after}{right_bystander_window}{right_context_close}"
    combinations = [seq for seq in combinations if seq != edited_focus]

    shift = (3 - (len(left_context_start) - ORF_start) % 3) % 3

    combinations_in_frame = [seq[shift:len(seq)-((len(seq)-shift)%3)] for seq in combinations]
    big_sequence = ''.join(combinations_in_frame)
    translated_big_sequence = str(Seq(big_sequence).translate())

    seq_length = len(combinations_in_frame[0]) // 3
    combinations_AA = [translated_big_sequence[i:i+seq_length] for i in range(0, len(translated_big_sequence), seq_length)]
    edited_focus_sequence = edited_focus[shift:len(edited_focus)-((len(edited_focus)-shift)%3)]
    edited_nobystander_focus_AA = str(Seq(edited_focus_sequence).translate())
    print(f"Edited focus sequence: {edited_focus_sequence}")
    print(f"Edited focus AA: {edited_nobystander_focus_AA}")
    print("--> check whether this AA stretch corresponds to your edited outcome (if you have an exon). If it does not match, check again whether your PRIDICT input sequence is in-frame")
    edited_nobystander_focus_AA_position = [i for i, x in enumerate(combinations_AA) if x == edited_nobystander_focus_AA]
    combinations_filtered = [combinations[i] for i in edited_nobystander_focus_AA_position]
    combinations_filtered_focus_only = [x[shift:len(x)-((len(x)-shift)%3)] for x in combinations_filtered]
    combinations_filtered_AA = [str(Seq(x).translate()) for x in combinations_filtered_focus_only]
    # print(combinations_filtered)
    # print(combinations_AA)

    return create_silent_bystander_df(combinations_filtered,combinations_filtered_focus_only,combinations_filtered_AA,  left_context_start, left_context_close, left_bystander_window, edit_before, right_bystander_window, right_context_close, right_context_end, name, edited_focus_sequence, edited_nobystander_focus_AA)

def bystander_input_generator(inputpath, inputfilename, outputpath, outputfilename, bystander_window, close_context_len, minimum_flanking):
    df = pd.read_csv(inputpath+inputfilename)
    all_dfs = []
    for _, row in df.iterrows():
        name = row["Name"]
        print(name)
        ORF_start = row["in_frame"]
        if ORF_start == 'yes':
            ORF_start = 0
        else:
            raise ValueError('ORF start is not in frame! Please check your input sequence. Input sequence has to be in-frame.')
        pridict_input_original = row.pridict_input
        silentbystanderdf = bystander_creation_for_pridict(pridict_input_original, ORF_start, name, bystander_window, close_context_len, minimum_flanking)
        print(f'{len(silentbystanderdf)} silent bystander sequences created for {name}')
        print()
        all_dfs.append(silentbystanderdf)

    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(outputpath+outputfilename)

    return df, final_df

In [9]:
x = 'agc(g/c)TAG'
x.upper()

'AGC(G/C)TAG'

### Manual mode:

In [11]:
### Required variables (adapt to your needs)
name = 'test1_cftr'
pridict_input_original = 'TGGACAGAAACAAAAAAACAATCTTTTAAACAGACTGGAGAGTTTGGGGAAAAAAGGAAGAATTCTATTCTCAATCCAATCAACTCTATACGAAAATTTTCCATTGTGCAAAAGACTCCCTTACAAATGAATGGCATCGAAGAGGATTCT(G/C)ATGAGCCTTTAGAGAGAAGGCTGTCCTTAGTACCAGATTCTGAGCAGGGAGAGGCGATACTGCCTCGCATCAGCGTGATCAGCACTGGCCCCACGCTTCAGGCACGAAGGAGGCAGTCTGTCCTGAACCTGATGACACACTCAGTTAACC'
pridict_input_original = 'ttttattttgcttgcttattttgttctattttattttattaaacagagaacggacactctttgaagtctctcatgaccactacagatgaacccaatctaccaacttcccaacagtccatacaatattagaagatgtttacattttgatggagg(T/G)aaacaaacctaaactatggtttgaatgactaagaaataacatttgatgagcttattagagaagtgtatattttgtggccacaatgtaggtttgatgtagttcagtttggacatatgcttgattttcagggcatcaaaaatttaaagttgatat'
# pridict_input_original = 'TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAGAGAACGGACACTCTTTGAAGTCTCTCATGACCACTACAGATGAACCCAATCTACCAACTTCCCAACAGTCCATACAATATTAGAAGATGTTTACATTTTGATGGAGG(T/G)AAACAAACCTAAACTATGGTTTGAATGACTAAGAAATAACATTTGATGAGCTTATTAGAGAAGTGTATATTTTGTGGCCACAATGTAGGTTTGATGTAGTTCAGTTTGGACATATGCTTGATTTTCAGGGCATCAAAAATTTAAAGTTGATAT'
###

### default variables (do not change unless you want to adapt the script)
ORF_start = 0
minimum_flanking = 94
bystander_window = 5
close_context_len = 5  # number of bases needed next to bystander window used for successful aminoacid translation at start and end of sequences
###

# run manual bystander creation:
silentbystanderdf = bystander_creation_for_pridict(pridict_input_original, ORF_start, name, bystander_window, close_context_len, minimum_flanking)

Edited focus sequence: TGATGGAGGgAAACAAAC
Edited focus AA: *WRETN
--> check whether this AA stretch corresponds to your edited outcome (if you have an exon). If it does not match, check again whether your PRIDICT input sequence is in-frame


In [12]:
# preview of silentbystanderdf
silentbystanderdf.head(10)

,sequence_name,editseq,original_edit_length,final_edit_length_with_bystander,total_nr_of_base_changes,bystander_focus_sequence,bystander_focus_AA,editedonly_focus_sequence,editedonly_focus_AA
0,test1_cftr_GT_ag,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,2,2,TGATGGAGagAAACAAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
1,test1_cftr_GTAAACA_agAAACt,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,7,3,TGATGGAGagAAACtAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
2,test1_cftr_GTAAACA_agAAACg,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,7,3,TGATGGAGagAAACgAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
3,test1_cftr_GTAAACA_agAAACc,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,7,3,TGATGGAGagAAACcAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
4,test1_cftr_GTAA_agAg,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,4,3,TGATGGAGagAgACAAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
5,test1_cftr_GTAAACA_agAgACt,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,7,4,TGATGGAGagAgACtAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
6,test1_cftr_GTAAACA_agAgACg,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,7,4,TGATGGAGagAgACgAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
7,test1_cftr_GTAAACA_agAgACc,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,7,4,TGATGGAGagAgACcAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
8,test1_cftr_TAAACA_gAAACt,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,6,2,TGATGGAGGgAAACtAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN
9,test1_cftr_TAAACA_gAAACg,TTTTATTTTGCTTGCTTATTTTGTTCTATTTTATTTTATTAAACAG...,1,6,2,TGATGGAGGgAAACgAAC,*WRETN,TGATGGAGGgAAACAAAC,*WRETN


### Batch mode:

In [22]:
minimum_flanking = 94
bystander_window = 5
close_context_len = 5  # number of bases needed next to bystander window used for successful aminoacid translation at start and end of sequences

inputpath = './input/'
inputfilename = 'input_testfile.csv'
inputfilename = 'input_lepr.csv'
outputpath = './output/'
outputfilename = 'outputfile.csv'
outputfilename = 'outputfile_lepr.csv'

# check input_testfile.csv for details about formatting the input file

# run bystander input generator
inputfiledf, outputfiledf = bystander_input_generator(inputpath, inputfilename, outputpath, outputfilename, bystander_window, close_context_len, minimum_flanking)

# continue with running PRIDICT2 (outside of this notebook) with the outputfile.csv as input file

LEPR_ORF1fw
Edited focus sequence: TGATGGAGGgAAACAAAC
Edited focus AA: *WRETN
--> check whether this AA stretch corresponds to your edited outcome (if you have an exon). If it does not match, check again whether your PRIDICT input sequence is in-frame
47 silent bystander sequences created for LEPR_ORF1fw

LEPR_ORF2fw
Edited focus sequence: GATGGAGGgAAACAAACC
Edited focus AA: DGGKQT
--> check whether this AA stretch corresponds to your edited outcome (if you have an exon). If it does not match, check again whether your PRIDICT input sequence is in-frame
7 silent bystander sequences created for LEPR_ORF2fw

LEPR_ORF3fw
Edited focus sequence: TTGATGGAGGgAAACAAACCT
Edited focus AA: LMEGNKP
--> check whether this AA stretch corresponds to your edited outcome (if you have an exon). If it does not match, check again whether your PRIDICT input sequence is in-frame
15 silent bystander sequences created for LEPR_ORF3fw

LEPR_ORF1rv
Edited focus sequence: GGTTTGTTTcCCTCCATC
Edited focus AA: GLFPS

### Summarize PRIDICT2.0 predictions of silent bystanders after running PRIDICT2.0
- Only run this after you ran PRIDICT2.0 with the output batch file created above. 

- The code below summarizes all the predictions with different silent bystanders in one file, sorts this by K562 score and saves it as summary prediction file.

- For MMR- context, change "sort_value" from "K562" to "HEK".

- From this summary file, we suggest to take e.g. the top 5 pegRNAs and test these in your experimental setup

In [ ]:
### Summarize PRIDICT2 predictions:
pridict2_predictions_folder = '../predictions/'
summary_prediction_output_folder = './summarized_silent_pridict2_predictions/'
sort_value = 'K562' # change to "HEK" for sorting to MMR-deficient cell line prediction

# filelist of all .csv files in pridict2_predictions_folder:
filelist = [f for f in os.listdir(pridict2_predictions_folder) if f.endswith('pegRNA_Pridict_full.csv')]

for index, row in inputfiledf.iterrows():
    sequence_name = row['Name']
    # get all files in filelist that start with the sequence_name
    sequence_files = [f for f in filelist if f.startswith(sequence_name)]
    # read all files and concatenate them
    all_files = []
    for file in sequence_files:
        all_files.append(pd.read_csv(pridict2_predictions_folder+file))
    all_files_df = pd.concat(all_files, ignore_index=True)
    # sort all_files_df by column "PRIDICT2_0_editing_Score_deep_..." (from highest to lowest)
    all_files_df = all_files_df.sort_values(by='PRIDICT2_0_editing_Score_deep_'+sort_value, ascending=False)
    # save concatenated file
    all_files_df.to_csv(summary_prediction_output_folder+sequence_name+'_all_silent_predictions.csv')